In [ ]:
import sys
!{sys.executable} -m pip install leafmap
!{sys.executable} -m pip install geopandas

1. Data Manipulation (5 points): Write a Python function that takes a list of lists representing points (latitude, longitude) and returns the average latitude and average longitude.

In [ ]:
xylist1 = [84.62, 63.11] #lists are formatted (lat,long)
xylist2 = [89.88, 65.89]
xylist3 = [78.93, 71.23]
##These lists are only an example, they could be data drawn from a file and formatted into lists

xymaster = [xylist1, xylist2, xylist3] #same here, if drawing from a file this list would have to automate the inputs rather tha just listing names
lat_sum = 0
long_sum = 0

for point in xymaster:
  lat_sum += point[0]
lat_avg = lat_sum / len(xymaster)

for point in xymaster:
  long_sum += point[1]
long_avg = long_sum / len(xymaster)

print(f'Average Latitude: {lat_avg}\nAverage Longitude: {long_avg}')

2. Conditional Statements (5 points): Write a Python program that reads a CSV file containing elevation data for points. The program should then identify and print the points with an elevation above a user-specified threshold.

In [ ]:
import os
import leafmap

elev_csv = 'https://raw.githubusercontent.com/wdwhitehead/Whitehead_Geog312_Final/main/elevation.csv'
df = leafmap.csv_to_df(elev_csv)
index = 0
message = 'Pick an elevation threshold. Points above the threshold will be displayed.'
thold = input(message)
thold = int(thold)
try:
    for point in elev_csv:
        elev = df['elevation'].loc[df.index[index]]
        if elev >= thold:
            print(f'{elev}')
        index += 1
except IndexError: #this is dumb but I didn't know how to stop it from giving this error so I'm working around it
        print('Done!')

3. Loops (5 points): Write a Python program that iterates through a dictionary containing countynames and their corresponding population densities. The program should calculate and print the total population for all counties. (Hint: Use a random number generator or google the population of your chosen cities to create the values for the population density. Use 10 different city names.)

In [ ]:
density_dict = { #pop data from tennessee-demographics.com, values are pop over sq miles
    'Shelby': 1200.742953732458,
    'Davidson': 1404.9639902386764,
    'Knox': 973.1685720469885,
    'Hamilton': 690.7471931862176,
    'Rutherford': 582.244575045208,
    'Williamson': 447.67421901819426,
    'Montgomery': 437.43833228235474,
    'Sumner': 385.03730286146,
    'Sullivan': 389.055544803561,
    'Wilson': 277.7622059106914
}
area_dict = { #In sq miles, data from usa.com
    'Shelby': 763.17,
    'Davidson': 504.03,
    'Knox': 508.21,
    'Hamilton': 542.43,
    'Rutherford': 619.36,
    'Williamson': 582.60,
    'Montgomery': 539.18,
    'Sumner': 529.45,
    'Sullivan': 413.36,
    'Wilson': 570.83
}
index = 0
for key, value in density_dict.items():
  population = value * (area_dict[key])
  population = int(population)
  index += 1
  print(f'{key} County Population: {population}')


4. Functions with Arguments (5 points): Write a Python function that takes a shapefile path as input and returns its area in square kilometers.

In [ ]:
import leafmap

message = "Please enter your shapefile's filepath:   " #paste "https://github.com/giswqs/leafmap/raw/master/examples/data/countries.zip"
in_shp = input(message)

m = leafmap.Map(center=[0, 0], zoom=2)
m.add_vector(in_shp, layer_name="shapefile")
leafmap.vector_area(in_shp, 'km2')

5. Error Handling (5 points): Write a Python program that attempts to read a raster file. If the file is not found or invalid, the program should print a helpful error message and gracefully exit

In [ ]:
#I didn't have this program actually do anything with the file, just read it.
#I also know that .tif isn't the only raster file extension, but this would be insanely long if I included every single one, so I assume that is not the intention.
raster_file = '/content/TN_Knoxville_147959_1978_24000_geo.tif'
if raster_file.endswith('.tif') == True: #this is a super convenient method. It's neat that it exists
  try:
    with open(raster_file) as file_object:
      contents = file_object.read()
  except FileNotFoundError:
    print(f'{raster_file} not found.')
else:
  print(f'{raster_file} is invalid. Please input a .tif file.')

1. Problem-Solving (10 points): You are tasked with creating a program to identify areas suitable for building a new solar farm. You have access to datasets for land cover, slope, and solar radiation. Describe the workflow of your program, including data preparation, analysis steps, and final output.

Primarily, it will be useful to identify 3 things:
1.) A large area with low to zero slope
2.) An area with high solar radiation
3.) An area that will take low amounts of pre-construction preparation, such as a clearing rather than a forest (but this is a lower priority)


Data prep:

We can first work with our slope raster to only select raster cells that have a slope of 1.0 degrees or less. We will reclass all cells that have 0-1.0 degrees slope as "1", and everything else as "0" using the "reclass" tool from Whitebox.

Then, we will perform the same workflow, only using the solar radiation raster. I don't really know what acceptable solar radiation would be, but whatever that value is, anything greater than or equal to it will be assigned as "1".

For our last raster, we will take the land cover raster and reclass plains, farmland, and other clear areas as "2". Water and other types of land cover that cannot be built on get reclassed to "0". Everything else gets "1".


Analysis:

We will use whitebox's raster_calculator to multiply our 0/1 solar raster, our 0/1 slope raster, and our 0/1/2 land cover raster. We will be left with a raster covered in values of 0, 1, and 2.


Output:

Our resulting 0/1/2 raster contains the suitability of each cell. 0 is not suitable for solar construction. 1 is suitable, but not ideal. 2 is ideal. Visual examination will provide large areas of 2 and 1 cells, these areas will be most suitable for solar farms.

2. Critical Thinking (10 points): You are given a shapefile of earthquake epicenters and a raster file of population density. Explain how you would use Python and GIS libraries to identify the areas most vulnerable to earthquake damage, considering both population density and proximity to epicenters.

Using the buffer tool in whitebox, you could create a buffer of some unspecified length for each epicenter.

Let's assume we have some sort of volatility field or something similar for each point. You could have a scaling distance on the buffer based on how volatile the epicenter is. (if we don't have that data, you could just have each buffer be 10 miles or something. I really don't know enough about earthquakes to decide that, though.)

You could then do zonal statistics on the raster cells inside the buffer. Perhaps take the average value of population density for each epicenter, and then apply that value to a new field on the shapefile for each point.

We could then display each point scaling with color and/or size based on that average population density value.

3. Critical Thinking (10 points): You are tasked with creating a visualization that shows the impacts of climate change on Tennessee agricultural production. You are given a dataset for precipitation, temperature, and crop output. How would you go about visualizing this in python and what analyses would need to be done to measure these impacts?

This would be a great use for the geemap timelapse feature. I am not confident how easy it is to import your own datasets rather than using the imagecollections already specified in the tool, however.

Using your timelapse, you could display all three maps simultaneously, allowing users to examine each timelapse on their own.

One way you could create some additional analysis to add to a table next to the visualization would be to create yearly  imagecollections with your available datasets to perform descriptive statistics on the collection. You could show how each year average values change. This is rather dependent on the data in hand, however. If you only have 1 image per year already including averaged values, you wouldn't need an imagecollection.

4. Geospatial Analysis (10 points): You have a shapefile of national parks and a raster file of deforestation rates. Design a Python program to calculate the total area of deforestation within each national park and identify the park with the highest deforestation rate.

One way to calculate the total area deforested is by using the zonal statistics by group tool to take a percentage of the area of each polygon covered by different values in the raster. Taking these numbers, we can see which polygons have the highest percentage of coverage by deforestation. With this list of values, we can use MAX to determine the highest percent coverage.

1.Shapefile Visualization (10 points): Write a Python program to read a shapefile. Create a map
showing the data points as colored polygons (anything other than circles). Each data point should
be colored based on its magnitude (e.g., green for small (low), yellow for medium (average), red
for large (high)).


In [ ]:
!pip install geopandas
!pip install shapely 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colormaps

#I cannot figure out the way to get my shapefiles to run straight off of GitHub. 
#If you download the "pointmag.shp" file off of my repository and fix the path, this code block works perfectly.
shapefile_path = '/content/pointmag.shp' 

gdf = gpd.read_file(shapefile_path)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

gdf.plot(column="Magnitude", ax=ax, cmap='RdYlGn', edgecolor='k',marker='D')

ax.set_title('Magnitude Points in Tennessee')
ax.set_axis_off()

plt.show()

2. Raster Processing and Visualization (10 points): Write a Python program to read a raster file of
precipitation data and apply a colormap to visualize the temperature variations. Add a legend to
the map and ensure clear labels for axes and title.
*Hint: Use the precipitation dataset already provided to you.

In [ ]:
#I cannot find the precipitation dataset anywhere

3. Cartographic Design (10 points): Design a map showing the distribution of solar farms in
west, middle, and east Tennessee. Create a raster layer that summarizes the total energy
generated from the solar farms in each of these regions (it can be one raster layer). Discuss which
region has the most solar production and why.

In [ ]:
#I spent 2 hours trying to figure this out and I do not know how I should make that raster

Bonus Question (10 points):
Develop a Python program that automates a repetitive GIS task you encountered during your
project or coursework. Explain the task, your approach, and the benefits of automation.

One task I do all the time at my job is assigning points identification numbers in a new field based on the category in a different field.

It's under NDA, but I can obfuscate in my example. "category" is my input, and "new field" is what my output is supposed to look like.

category : Red11
new field: 11

"new field" has to be an integer, not a string, so we can do math operations. Normally I do this manually for each point in the shapefile. It is really boring, takes probably 2-3 hours a week, and I've been thinking about optimizing it. Python would be perfect.


I can use python3 code in the calculate field function of ArcGIS pro to cut off the end of the string in the category field by selecting the final two characters, and then classify that variable as an integer. I can then put that variable in the "calculate field" section, and it will iterate through each item in the list, doing the work automatically. 

I'm actually going to do this tomorrow at my job. This will save me a bunch of time in the future, and I'll be able to pass off the script to my coworkers as well. Between the 3 of us, it could save 6-10 hours per week.